In [0]:
#Move the file from Workspace to DBFS

dbutils.fs.cp("file:/Workspace/Shared/employee_updates.csv", "dbfs:/FileStore/employee_updates.csv")

True

In [0]:
#Convert employee CSV data to Delta format
df_employee = spark.read.format("csv").option("header", "true").load("/FileStore/employee_data.csv")

df_employee.write.format("delta").mode ("overwrite").save("/delta/employee_data")

In [0]:
# Convert employee updates CSV data to Delta format
df_employee_updates = spark.read.format("csv").option("header", "true").load("/FileStore/employee_updates.csv")

df_employee_updates.write.format("delta").mode("overwrite").save("/delta/employee_updates")

In [0]:
# Load Delta tables
df_employee = spark.read.format("delta").load("/delta/employee_data")
df_employee_updates = spark.read.format("delta").load("/delta/employee_updates")

In [0]:
# Create temporary views for SQL operations with corrected view names
df_employee.createOrReplaceTempView("delta_employee") 

df_employee_updates.createOrReplaceTempView("employee_updates")

In [0]:
#### CELL 4 ####
spark.sql("""
    MERGE INTO delta_employee AS target
    USING employee_updates AS source
    ON target.EmployeeID = source.EmployeeID
    WHEN MATCHED THEN UPDATE SET
        target.Salary = source.Salary,
        target.Department = source.Department
    WHEN NOT MATCHED THEN INSERT 
    (
        EmployeeID, Name, Department, JoiningDate, Salary
    ) VALUES (
        source.EmployeeID, source.Name, source.Department, source.JoiningDate, source.Salary
    )
""")

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
#Query the Delta table to check if the data was updated or inserted correctly 
spark.sql("SELECT * FROM delta_employee").show()

+----------+-------------+----------+-----------+------+
|EmployeeID|         Name|Department|JoiningDate|Salary|
+----------+-------------+----------+-----------+------+
|      1002|   Jane Smith|        IT| 2020-03-10| 62000|
|      1003|Emily Johnson|   Finance| 2019-07-01| 70000|
|      1004|Michael Brown|        HR| 2018-12-22| 54000|
|      1005| David Wilson|        IT| 2021-06-25| 58000|
|      1006|  Linda Davis|   Finance| 2020-11-15| 67000|
|      1007| James Miller|        IT| 2019-08-14| 65000|
|      1008|Barbara Moore|        HR| 2021-03-29| 53000|
|      1001|     John Doe|        HR| 2021-01-15| 58000|
|      1009|  Sarah Adams| Marketing| 2021-09-01| 60000|
|      1010|  Robert King|        IT| 2022-01-10| 62000|
+----------+-------------+----------+-----------+------+

